# Calculate indices from MULTISPECTRAL image

#### Import libraries and read hyperspectral image (.tiff)

In [1]:
from math import log
import numpy as np
import pandas as pd
import rasterio
from rasterio.plot import show
from rasterio.transform import rowcol
import matplotlib.pyplot as plt
from pyproj import Proj, transform
import openpyxl
import shutil

# Path to the .tiff file
working_dir = "C:/Users/User/OneDrive - UVa/1_ASIGNATURAS/TFM/1_DataProcessing/"
tiff_file = working_dir + "2_SatellitalImages/1_MLT/S2_post_clip.tif"

#### Open multispectral image and classify wavelengths

In [2]:
# Sentinel 2 bands
band_lst = [
    (1, "Blue", "B2", 450, 520),
    (2, "Green", "B3", 540, 570),
    (3, "Red", "B4", 650, 680),
    (4, "Red Edge", "B5", 690, 710),
    (5, "Red Edge", "B6", 730, 740),
    (6, "Red Edge", "B7", 770, 790),
    (7, "NIR", "B8", 780, 900),
    (8, "NIR", "B8A", 850, 870),
    (9, "SWIR 1", "B11", 1560, 1650),
    (10, "SWIR 2", "B12", 2100, 2280),
]

# Display band information for the 10 bands assumed to be in your TIFF file
print(f"{'Band':<5} {'Name':<25} {'Wavelength (nm)':<15}")
print("="*50)
for band in band_lst:
    print(f"{band[2]:<5} {band[1]:<25} {band[3]} - {band[4]:<15}")

Band  Name                      Wavelength (nm)
B2    Blue                      450 - 520            
B3    Green                     540 - 570            
B4    Red                       650 - 680            
B5    Red Edge                  690 - 710            
B6    Red Edge                  730 - 740            
B7    Red Edge                  770 - 790            
B8    NIR                       780 - 900            
B8A   NIR                       850 - 870            
B11   SWIR 1                    1560 - 1650           
B12   SWIR 2                    2100 - 2280           


#### Vegetation Indices Equations

In [3]:
# Normalized Difference Vegetation Index: NDVI
def calculate_ndvi(red, nir):
    ndvi = (nir - red) / (nir + red)
    return ndvi

# Normalized Burn Ratio: NBR
def calculate_nbr(nir, swir2):
    nbr = (nir - swir2) / (nir + swir2)
    return nbr

# Normalized Difference Moisture Index: NDMI
def calculate_ndmi(nir, swir1):
    ndmi = (nir - swir1) / (nir + swir1)
    return ndmi

# Difference Vegetation Index: DVI
def calculate_dvi(red, nir):
    dvi = nir - red
    return dvi

# Green Difference Vegetation Index: DVIGRE o GDVI
def calculate_dvigre(green, nir):
    dvigre = nir - green
    return dvigre

# Red Difference Vegetation Index: DVIRED
def calculate_dvired(redge, nir):
    dvired = nir - redge
    return dvired

# Chlorophyll Index With Red Edge: CIREDGE
def calculate_ciredge(redge, nir):
    ciredge = (nir / redge) - 1
    return ciredge

# Chlorophyll Index With Green: CIGREEN
def calculate_cigreen(green, nir):
    cigreen = (nir / green) - 1
    return cigreen

# Infrared Percentage Vegetation Index: IPVI
def calculate_ipvi(red, nir):
    ipvi = nir / (nir + red)
    return ipvi

# Near-Infrared Reflectance of Vegetation: NIRV
def calculate_nirv(red, nir):
    nirv = nir * ((nir - red) / (nir + red))
    return nirv

# Modified Non-Linear Index: MNLI
def calculate_mnli(nir, red): 
    mnli = 1.5 * (nir ** 2 - red) / (nir ** 2 + red + 0.5)
    return mnli

# Non-Linear Index: NLI
def calculate_nli(nir, red): 
    nli = (nir ** 2 - red) / (nir ** 2 + red)
    return nli

# Wide Dynamic Range Vegetation Index: WDRVI
def calculate_wdrvi(nir, red): 
    wdrvi = (0.2 * nir - red) / (0.2 * nir + red)
    return wdrvi

##########################################################################
# Normalized Difference Red Edge Index: NDRE
def calculate_ndre(redge, nir):
    ndre = (nir - redge) / (nir + redge)
    return ndre

# Burn Area Index: BAI
def calculate_bai(redge, nir):
    bai = 1 / (((redge - 0.1) ** 2) + ((nir - 0.06) ** 2))
    return bai

# Structural Vegetation Index: SVI
def calculate_svi(redge, red):
    svi = (redge - red) / (redge + red)
    return svi

# Modified Red-Edge Simple Ratio: MRESR
def calculate_mresr(redge, nir):
    mresr = nir / redge
    return mresr

In [4]:
# Three-Band Difference Vegetation Index: TBDVI
def calculate_tbdvi(red, nir, swir1):
    tbdvi = nir * (red - swir1) / 2
    return tbdvi

# Enhanced Vegetation Index: EVI
def calculate_evi(blue, red, nir):
    evi = 2.5 * ((nir - red)/(nir + 6 * red - 7.5 * blue + 1))
    return evi

# Excess Green Index: EXG2
def calculate_exg2(blue, green, red):
    exg2 = 2 * green - red - blue
    return exg2

# Red Light Normalized Value: NRI
def calculate_nri(blue, green, red):
    nri = red / (red + green + blue)
    return nri

# Green Light Normalized Value: NGI
def calculate_ngi(blue, green, red):
    ngi = green / (red + green + blue)
    return ngi

# Green Normalized Difference Vegetation Index: GNDVI
def calculate_gndvi(green, redge, nir):
    gndvi = (nir - green) / (nir + redge)
    return gndvi

# Enhances Normalized Difference Vegetation Index: ENDVI
def calculate_endvi(redge, green, blue): 
    endvi = (redge + green - 2 * blue) / (redge + green + 2 * blue)
    return endvi

# Modified Red Green- Blue Vegetation Index: MRGBVI
def calculate_mrgbvi(redge, green, blue): 
    mrgbvi = (redge + 2 * green - 2 * blue) / (redge + 2 * green + 2 * blue)
    return mrgbvi

# Nitrogen Reflectance Index: NREI
def calculate_nrei(green, redge, nir):
    nrei = redge / (redge + nir + green)
    return nrei

##########################################################################
# Red Edge Position Index: REP
def calculate_rep(red, redge, nir):
    rep = 700 + 40 * ((red + nir)/(2 - redge))
    return rep

In [5]:
def normalized_band (red, green, blue):
    R = red / (red + green + blue)
    G = green / (red + green + blue)
    B = blue / (red + green + blue)
    return R, G, B

# Excess Green Index: EXG
def calculate_exg(red, green, blue):
    R,G,B = normalized_band(red, green, blue)
    exg = 2 * G - R - B
    return exg

# Excess Blue Index: EXB
def calculate_exb(red, green, blue):
    R,G,B = normalized_band(red, green, blue)
    exb = 1.4 * R - G
    return exb

# Excess Red Index: EXR
def calculate_exr(red, green, blue):
    R,G,B = normalized_band(red, green, blue)
    exr = 1.4 * B - G
    return exr

# Red / Green Ratio: RGR
def calculate_rgr(red, green, blue):
    R,G,B = normalized_band(red, green, blue)
    rgr = R / G
    return rgr

# Blue / Green Ratio: BGR
def calculate_bgr(red, green, blue):
    R,G,B = normalized_band(red, green, blue)
    bgr = B / G
    return bgr

# Normalized Green-Red Difference Index: NGRDI
def calculate_ngrdi(red, green, blue):
    R,G,B = normalized_band(red, green, blue)
    ngrdi = (G - R) / (G + R)
    return ngrdi

# Normalized Green-Blue Difference Index: NGBDI
def calculate_ngbdi(red, green, blue):
    R,G,B = normalized_band(red, green, blue)
    ngbdi = (G - B) / (G + B)
    return ngbdi

# Modified Green-Red Vegetation Index: MGRVI
def calculate_mgrvi(red, green, blue):
    R,G,B = normalized_band(red, green, blue)
    mgrvi = (G**2 - R**2) / (G**2 + R**2)
    return mgrvi

# Red Green- Blue Vegetation Index: RGBVI
def calculate_rgbvi(red, green, blue):
    R,G,B = normalized_band(red, green, blue)
    rgbvi = (G**2 - R * B) / (G**2 + R * B)
    return rgbvi

# Green Leaf Index: GLI
def calculate_gli(red, green, blue):
    R,G,B = normalized_band(red, green, blue)
    gli = (2*G - R - B) / (2*G + R + B)
    return gli

# Color Index of Vegetation Extraction: CIVE
def calculate_cive(red, green, blue):
    R,G,B = normalized_band(red, green, blue)
    cive = 0.441*R - 0.881*G + 0.385*R + 18.78745
    return cive

# Excess Green Minus Excess Red Index: ExGR
def calculate_exgr(red, green, blue):
    exg = calculate_exg(red, green, blue)
    exr = calculate_exr(red, green, blue)
    exgr = exg - exr
    return exgr

# Improved Red Green- Blue Vegetation Index: IRGBVI
def calculate_irgbvi(red, green, blue):
    R,G,B = normalized_band(red, green, blue)
    irgbvi = (5*G**2 - 2*R**2 - 5*B**2) / (5*G**2 + 2*R**2 + 5*B**2)
    return irgbvi

#### Dictionaries of vegetation indices bands

In [ ]:
# Classed bands to calculate vegetatio index
vi_classed_band_dict = {
    "NDVI": ("Red", "NIR"),
    "NBR": ("NIR", "SWIR 2"),
    "NDMI": ("NIR", "SWIR 1"),
    "DVI": ("Red", "NIR"),
    "DVIGRE": ("Green", "NIR"),
    "DVIRED": ("Red Edge", "NIR"),
    "CIREDGE": ("Red Edge", "NIR"),
    "CIGREEN": ("Green", "NIR"),
    "IPVI": ("Red", "NIR"),
    "NIRV": ("Red", "NIR"),
    "MNLI": ("NIR", "Red"),
    "NLI": ("NIR", "Red"),
    "WDRVI": ("NIR", "Red"),
    "NDRE": ("Red Edge", "NIR"),
    "BAI": ("Red Edge", "NIR"),
    "SVI": ("Red Edge", "Red"),
    "MRESR": ("Red Edge", "NIR"),
}

vi_classed_band_dict2 = {
    "TBDVI": ("Red", "NIR", "SWIR 1"),
    "EVI": ("Blue", "Red", "NIR"),
    "EXG2": ("Blue", "Green", "Red"),
    "NRI": ("Blue", "Green", "Red"),
    "NGI": ("Blue", "Green", "Red"),
    "GNDVI": ("Green", "Red Edge", "NIR"),
    "ENDVI": ("Red Edge", "Green", "Blue"),
    "MRGBVI": ("Red Edge", "Green", "Blue"),
    "NREI": ("Green", "Red Edge", "NIR"),
    "EXG": ("Red", "Green", "Blue"),
    "EXB": ("Red", "Green", "Blue"),
    "EXR": ("Red", "Green", "Blue"),
    "RGR": ("Red", "Green", "Blue"),
    "BGR": ("Red", "Green", "Blue"),
    "NGRDI": ("Red", "Green", "Blue"),
    "NGBDI": ("Red", "Green", "Blue"),
    "MGRVI": ("Red", "Green", "Blue"),
    "RGBVI": ("Red", "Green", "Blue"),
    "GLI": ("Red", "Green", "Blue"),
    "CIVE": ("Red", "Green", "Blue"),
    "EXGR": ("Red", "Green", "Blue"),
    "IRGBVI": ("Red", "Green", "Blue"),
    "REP": ("Red", "Red Edge", "NIR"),
}

vi_classed_band_dict3 = {
    # "NewMltVeg": ("NIR", "SWIR 1", "Red Edge", "Red Edge"),
}

#### Read the excel file

In [7]:
# Read the Excel file
excel_file = working_dir + "CulebraPointsCBI.xlsx"
points = pd.read_excel(excel_file)

In [ ]:
# Initialize lists to store results
cb_results_lst = 0
cb2_results_lst = 0
cb3_results_lst = 0
fb_results_lst = 0
vb_results_lst = 0

### FOR CLASSED BANDS

#### Combination of bands of CLASSED BANDS

In [ ]:
def bands_combinations(tiff_file, band_lst, vi_classed_band_dict):
    for i in vi_classed_band_dict:
        print(i)
    vi_name = input("Insert the index name: ")
    combination_lst = []
    band_names = vi_classed_band_dict[vi_name]
    with rasterio.open(tiff_file) as src:
        for band_a in band_lst:
            if band_a[1] == band_names[0]:
                band_ax = band_a[0]
                for band_b in band_lst:
                    if band_b[1] == band_names[1]:
                        band_bx = band_b[0]
                        combination_lst.append((band_ax, band_bx))
    for i in combination_lst:
        print(i)
    return combination_lst, vi_name

def bands_combinations2(tiff_file, band_lst, vi_classed_band_dict2):
    for i in vi_classed_band_dict2:
        print(i)
    vi_name = input("Insert the index name: ")
    combination_lst2 = []
    band_names = vi_classed_band_dict2[vi_name]
    with rasterio.open(tiff_file) as src:
        for band_a in band_lst:
            if band_a[1] == band_names[0]:
                band_ax = band_a[0]
                for band_b in band_lst:
                    if band_b[1] == band_names[1]:
                        band_bx = band_b[0]
                        for band_c in band_lst:
                            if band_c[1] == band_names[2]:
                                band_cx = band_c[0]
                                combination_lst2.append((band_ax, band_bx, band_cx))
    for i in combination_lst2:
        print(i)
    return combination_lst2, vi_name

# def bands_combinations3(tiff_file, band_lst, vi_classed_band_dict3):
#     for i in vi_classed_band_dict3:
#         print(i)
#     vi_name = input("Insert the index name: ")
#     combination_lst3 = []
#     band_names = vi_classed_band_dict3[vi_name]
#     with rasterio.open(tiff_file) as src:
#         for band_a in band_lst:
#             if band_a[1] == band_names[0]:
#                 band_ax = band_a[0]
#                 for band_b in band_lst:
#                     if band_b[1] == band_names[1]:
#                         band_bx = band_b[0]
#                         for band_c in band_lst:
#                             if band_c[1] == band_names[2]:
#                                 band_cx = band_c[0]
#                                 for band_d in band_lst:
#                                     if band_d[1] == band_names[3]:
#                                         band_dx = band_d[0]
#                                         combination_lst3.append((band_ax, band_bx, band_cx, band_dx))
#     for i in combination_lst3:
#         print(i)
#     return combination_lst3, vi_name

#### Calculate vegetation index values for CLASSED BANDS

In [9]:
def get_vi_cb_image(tiff_file, vi_name, band_a, band_b):
    with rasterio.open(tiff_file) as dataset:
        band_1 = dataset.read(band_a)
        band_2 = dataset.read(band_b)
        if vi_name == "NDVI":
            vi_values = calculate_ndvi(band_1, band_2)
        elif vi_name == "NBR":
            vi_values = calculate_nbr(band_1, band_2)
        elif vi_name == "NDMI":
            vi_values = calculate_ndmi(band_1, band_2)
        elif vi_name == "DVI":
            vi_values = calculate_dvi(band_1, band_2)
        elif vi_name == "DVIGRE":
            vi_values = calculate_dvigre(band_1, band_2)
        elif vi_name == "DVIRED":
            vi_values = calculate_dvired(band_1, band_2)
        elif vi_name == "CIREDGE":
            vi_values = calculate_ciredge(band_1, band_2)
        elif vi_name == "CIGREEN":
            vi_values = calculate_cigreen(band_1, band_2)
        elif vi_name == "IPVI":
            vi_values = calculate_ipvi(band_1, band_2)
        elif vi_name == "NIRV":
            vi_values = calculate_nirv(band_1, band_2)
        elif vi_name == "MNLI":
            vi_values = calculate_mnli(band_1, band_2)
        elif vi_name == "NLI":
            vi_values = calculate_nli(band_1, band_2)
        elif vi_name == "WDRVI":
            vi_values = calculate_wdrvi(band_1, band_2)
        elif vi_name == "NDRE":
            vi_values = calculate_ndre(band_1, band_2)
        elif vi_name == "BAI":
            vi_values = calculate_bai(band_1, band_2)
        elif vi_name == "SVI":
            vi_values = calculate_svi(band_1, band_2)
        elif vi_name == "MRESR":
            vi_values = calculate_mresr(band_1, band_2)
        else:
            print("Insert other vegetation index name")
    return vi_values, dataset

def get_vi_cb_at_points(vi_values, dataset, points):
    veg_index_values = []
    for _, row in points.iterrows():
        name, x, y = row['Parcela'], row['Coord_X'], row['Coord_Y']
        row_idx, col_idx = rasterio.transform.rowcol(dataset.transform, x, y)
        veg_index_value = vi_values[row_idx, col_idx]
        veg_index_values.append((name, veg_index_value))
    return veg_index_values

def get_cb_comparative_points(tiff_file, band_lst, vi_classed_band_dict, points):
    cb_results_lst = []
    combination_lst, vi_name = bands_combinations(tiff_file, band_lst, vi_classed_band_dict)
    for paired_bands in combination_lst:
        band_a = paired_bands[0]
        band_b = paired_bands[1]
        vi_values, dataset = get_vi_cb_image(tiff_file, vi_name, band_a, band_b)
        veg_index_values = get_vi_cb_at_points(vi_values, dataset, points)
        cb_results_lst.append((vi_name, band_a, band_b, veg_index_values))
    return cb_results_lst

In [11]:
def get_vi_cb2_image(tiff_file, vi_name, band_a, band_b, band_c):
    with rasterio.open(tiff_file) as dataset:
        band_1 = dataset.read(band_a)
        band_2 = dataset.read(band_b)
        band_3 = dataset.read(band_c)
        if vi_name == "TBDVI":
            vi_values = calculate_tbdvi(band_1, band_2, band_3)
        elif vi_name == "EVI":
            vi_values = calculate_evi(band_1, band_2, band_3)
        elif vi_name == "EXG2":
            vi_values = calculate_exg2(band_1, band_2, band_3)
        elif vi_name == "NRI":
            vi_values = calculate_nri(band_1, band_2, band_3)
        elif vi_name == "NGI":
            vi_values = calculate_ngi(band_1, band_2, band_3)
        elif vi_name == "GNDVI":
            vi_values = calculate_gndvi(band_1, band_2, band_3)
        elif vi_name == "ENDVI":
            vi_values = calculate_endvi(band_1, band_2, band_3)
        elif vi_name == "MRGBVI":
            vi_values = calculate_mrgbvi(band_1, band_2, band_3)
        elif vi_name == "NREI":
            vi_values = calculate_nrei(band_1, band_2, band_3)
        elif vi_name == "EXG":
            vi_values = calculate_exg(band_1, band_2, band_3)
        elif vi_name == "EXB":
            vi_values = calculate_exb(band_1, band_2, band_3)
        elif vi_name == "EXR":
            vi_values = calculate_exr(band_1, band_2, band_3)
        elif vi_name == "RGR":
            vi_values = calculate_rgr(band_1, band_2, band_3)
        elif vi_name == "BGR":
            vi_values = calculate_bgr(band_1, band_2, band_3)
        elif vi_name == "NGRDI":
            vi_values = calculate_ngrdi(band_1, band_2, band_3)
        elif vi_name == "NGBDI":
            vi_values = calculate_ngbdi(band_1, band_2, band_3)
        elif vi_name == "MGRVI":
            vi_values = calculate_mgrvi(band_1, band_2, band_3)
        elif vi_name == "RGBVI":
            vi_values = calculate_rgbvi(band_1, band_2, band_3)
        elif vi_name == "GLI":
            vi_values = calculate_gli(band_1, band_2, band_3)
        elif vi_name == "CIVE":
            vi_values = calculate_cive(band_1, band_2, band_3)
        elif vi_name == "EXGR":
            vi_values = calculate_exgr(band_1, band_2, band_3)
        elif vi_name == "IRGBVI":
            vi_values = calculate_irgbvi(band_1, band_2, band_3)
        elif vi_name == "REP":
            vi_values = calculate_rep(band_1, band_2, band_3)
        # elif vi_name == "NewMltSoil":
        #     vi_values = calculate_new_vi_mlt_soil(band_1, band_2, band_3)
        # elif vi_name == "NewMltSite":
        #     vi_values = calculate_new_vi_mlt_site(band_1, band_2, band_3)
        else:
            print("Insert other vegetation index name")
    return vi_values, dataset

def get_cb2_comparative_points(tiff_file, band_lst, vi_classed_band_dict2, points):
    cb2_results_lst = []
    combination_lst2, vi_name = bands_combinations2(tiff_file, band_lst, vi_classed_band_dict2)
    for paired_bands in combination_lst2:
        band_a = paired_bands[0]
        band_b = paired_bands[1]
        band_c = paired_bands[2]
        vi_values, dataset = get_vi_cb2_image(tiff_file, vi_name, band_a, band_b, band_c)
        veg_index_values = get_vi_cb_at_points(vi_values, dataset, points)
        cb2_results_lst.append((vi_name, band_a, band_b, band_c, veg_index_values))
    return cb2_results_lst

#### Extract results of wavelength and vegetation index of CLASSED BANDS

In [18]:
cb_results_lst = get_cb_comparative_points(tiff_file, band_lst, vi_classed_band_dict, points)
for i in cb_results_lst:
    print(i)

BAI
SVI
MRESR
(4, 7)
(4, 8)
(5, 7)
(5, 8)
(6, 7)
(6, 8)
('MRESR', 4, 7, [('CBI_01', 1.2840623428858724), ('CBI_02', 1.4211618257261411), ('CBI_03', 1.4551351351351351), ('CBI_04', 1.6610703043022035), ('CBI_05', 1.0579290369297611), ('CBI_06', 1.0132219902574808), ('CBI_07', 1.0780619111709286), ('CBI_08', 1.0550284629981024), ('CBI_09', 1.0922535211267606), ('CBI_10', 1.0388349514563107), ('CBI_11', 1.0910307898259706), ('CBI_12', 1.1013557133634604), ('CBI_13', 1.10397946084724), ('CBI_14', 1.0737704918032787), ('CBI_15', 1.111695137976347), ('CBI_16', 1.0558069381598794), ('CBI_17', 1.0570054945054945), ('CBI_18', 1.0773558368495078), ('CBI_19', 1.0544522741832159), ('CBI_20', 1.141155449672424), ('CBI_21', 1.0310370931112793), ('CBI_22', 1.207227555053642), ('CBI_23', 2.019120458891013), ('CBI_24', 1.0598568640208197), ('CBI_25', 1.2699738903394255), ('CBI_26', 1.048502994011976), ('CBI_27', 1.1437234687689508), ('CBI_28', 1.0675990675990676), ('CBI_29', 1.0604265402843602), ('CBI_

In [20]:
cb2_results_lst = get_cb2_comparative_points(tiff_file, band_lst, vi_classed_band_dict2, points)
for i in cb2_results_lst:
    print(i)

REP
(3, 4, 7)
(3, 4, 8)
(3, 5, 7)
(3, 5, 8)
(3, 6, 7)
(3, 6, 8)
('REP', 3, 4, 7, [('CBI_01', 702.7657398228139), ('CBI_02', 702.7668605565162), ('CBI_03', 702.7483984424067), ('CBI_04', 702.948202162434), ('CBI_05', 701.7538226538024), ('CBI_06', 701.7541068002059), ('CBI_07', 701.9134453256729), ('CBI_08', 701.9944650311928), ('CBI_09', 701.8509622882809), ('CBI_10', 701.6916151341921), ('CBI_11', 701.9249266129536), ('CBI_12', 702.005344668615), ('CBI_13', 702.0325101594249), ('CBI_14', 701.8709617005337), ('CBI_15', 701.9770057485629), ('CBI_16', 701.6813056749518), ('CBI_17', 701.8445117193596), ('CBI_18', 701.8179549566411), ('CBI_19', 701.9682073245073), ('CBI_20', 702.2292863965924), ('CBI_21', 701.6624881261971), ('CBI_22', 702.3548230275848), ('CBI_23', 703.6289127762192), ('CBI_24', 701.9455946000844), ('CBI_25', 702.5374471496157), ('CBI_26', 702.0630049477046), ('CBI_27', 702.1737701325737), ('CBI_28', 702.2236846228574), ('CBI_29', 702.1425215348473), ('CBI_30', 701.773107

In [23]:
cb3_results_lst = get_cb3_comparative_points(tiff_file, band_lst, vi_classed_band_dict3, points)
for i in cb3_results_lst:
    print(i)

NewMltVeg
(7, 9, 4, 4)
(7, 9, 4, 5)
(7, 9, 4, 6)
(7, 9, 5, 4)
(7, 9, 5, 5)
(7, 9, 5, 6)
(7, 9, 6, 4)
(7, 9, 6, 5)
(7, 9, 6, 6)
(8, 9, 4, 4)
(8, 9, 4, 5)
(8, 9, 4, 6)
(8, 9, 5, 4)
(8, 9, 5, 5)
(8, 9, 5, 6)
(8, 9, 6, 4)
(8, 9, 6, 5)
(8, 9, 6, 6)
('NewMltVeg', 7, 9, 4, 4, [('CBI_01', -27846), ('CBI_02', -35389), ('CBI_03', -35927), ('CBI_04', -48854), ('CBI_05', -11384), ('CBI_06', -10752), ('CBI_07', -13885), ('CBI_08', -14501), ('CBI_09', -13371), ('CBI_10', -10028), ('CBI_11', -14802), ('CBI_12', -16393), ('CBI_13', -18929), ('CBI_14', -16114), ('CBI_15', -18613), ('CBI_16', -11253), ('CBI_17', -12557), ('CBI_18', -12613), ('CBI_19', -14364), ('CBI_20', -19993), ('CBI_21', -9610), ('CBI_22', -20826), ('CBI_23', -80900), ('CBI_24', -13545), ('CBI_25', -28677), ('CBI_26', -18461), ('CBI_27', -21483), ('CBI_28', -15805), ('CBI_29', -15184), ('CBI_30', -11817), ('CBI_31', -28908), ('CBI_32', -49493), ('CBI_33', -26209), ('CBI_34', -12935), ('CBI_35', -9993), ('CBI_36', -21938), ('CBI_37', 

### FOR RAW BANDS

#### Calculate vegetation index values for RAW BANDS

In [1]:
def get_rb_image(tiff_file, band_number):
    with rasterio.open(tiff_file) as dataset:
        band_values = dataset.read(band_number)
    return band_values, dataset

def get_rb_at_points(band_values, dataset, points):
    band_values_lst = []
    for _, row in points.iterrows():
        name, x, y = row['Parcela'], row['Coord_X'], row['Coord_Y']
        row_idx, col_idx = rasterio.transform.rowcol(dataset.transform, x, y)
        band_value = band_values[row_idx, col_idx]
        band_values_lst.append((name, band_value))
    return band_values_lst

def get_rb_comparative_points(tiff_file, band_lst, points):
    rb_results_lst = []

    for band_number in range(1,11):
        band_values, dataset = get_rb_image(tiff_file, band_number)
        band_values_lst = get_rb_at_points(band_values, dataset, points)
        rb_results_lst.append((band_lst[band_number-1][1], band_number, band_lst[band_number-1][3], band_lst[band_number-1][4], band_values_lst))

    return rb_results_lst

#### Extract results of wavelength and vegetation index of RAW BANDS

In [8]:
rb_results_lst = get_rb_comparative_points(tiff_file, band_lst, points)
for i in rb_results_lst:
    print(i)

('Blue', 1, 450, 520, [('CBI_01', 1498), ('CBI_02', 1433), ('CBI_03', 1408), ('CBI_04', 1346), ('CBI_05', 1290), ('CBI_06', 1256), ('CBI_07', 1338), ('CBI_08', 1385), ('CBI_09', 1309), ('CBI_10', 1269), ('CBI_11', 1284), ('CBI_12', 1316), ('CBI_13', 1380), ('CBI_14', 1309), ('CBI_15', 1338), ('CBI_16', 1234), ('CBI_17', 1288), ('CBI_18', 1256), ('CBI_19', 1382), ('CBI_20', 1434), ('CBI_21', 1263), ('CBI_22', 1358), ('CBI_23', 1366), ('CBI_24', 1394), ('CBI_25', 1371), ('CBI_26', 1369), ('CBI_27', 1380), ('CBI_28', 1505), ('CBI_29', 1474), ('CBI_30', 1317), ('CBI_31', 1261), ('CBI_32', 1315), ('CBI_33', 1402), ('CBI_34', 1416), ('CBI_35', 1231), ('CBI_36', 1345), ('CBI_37', 1444), ('CBI_38', 1286), ('CBI_39', 1376), ('CBI_40', 1383), ('CBI_41', 1471), ('CBI_42', 1272), ('CBI_43', 1358), ('CBI_44', 1335), ('CBI_45', 1354), ('CBI_46', 1372), ('CBI_47', 1507), ('CBI_48', 1407), ('CBI_49', 1303), ('CBI_50', 1303), ('CBI_51', 1332), ('CBI_52', 1337), ('CBI_53', 1282), ('CBI_54', 1372), ('CBI

### SAVE INDICES VALUES

#### Save vegetation indices values in an excel file (.xlsx)

In [14]:
def save_xlsx(input_file, results_lst):
    # Creamos un diccionario para almacenar los datos
    data_dict = {}

    for elm in results_lst:
        if results_lst == cb_results_lst:
            col_name = f"{elm[0]}_{elm[1]}_{elm[2]}"
            type_name = "classed_bands"
        elif results_lst == cb2_results_lst:
            col_name = f"{elm[0]}_{elm[1]}_{elm[2]}_{elm[3]}"
            type_name = "classed_bands"
        elif results_lst == cb3_results_lst:
            col_name = f"{elm[0]}_{elm[1]}_{elm[2]}_{elm[3]}_{elm[4]}"
            type_name = "classed_bands"
        elif results_lst == rb_results_lst:
            col_name = f"{elm[0]}_{elm[1]}_{elm[2]}-{elm[3]}nm"
            type_name = "raw_bands"
        data_dict[col_name] = [value for point, value in elm[len(elm)-1] ]

    # Convertimos el diccionario en un DataFrame
    df = pd.DataFrame(data_dict)

    # Copia el archivo original al nuevo archivo
    if type_name == "classed_bands":
        output_file = input_file[:-5] + "_mlt_" + type_name + "_" + results_lst[0][0] + ".xlsx"
    else:
        output_file = input_file[:-5] + "_mlt_" + type_name + ".xlsx"
    shutil.copyfile(input_file, output_file)

    # Abre el nuevo archivo
    wb = openpyxl.load_workbook(output_file)
    sheet = wb.active

    # Encuentra la última columna con datos en la hoja existente
    last_col = sheet.max_column

    # Escribe los datos del DataFrame en las columnas nuevas a partir de la última columna existente
    for col_idx, col_name in enumerate(df.columns, start=last_col + 1):
        sheet.cell(row=1, column=col_idx, value=col_name)
        for row_idx, value in enumerate(df[col_name], start=2):
            sheet.cell(row=row_idx, column=col_idx, value=value)

    # Guarda el archivo con las nuevas columnas
    wb.save(output_file)
    return output_file

In [21]:
## Options to save file
# save_xlsx(excel_file, cb_results_lst)
save_xlsx(excel_file, cb2_results_lst)
# save_xlsx(excel_file, cb3_results_lst)
# save_xlsx(excel_file, fb_results_lst)
# save_xlsx(excel_file, vb_results_lst)
# save_xlsx(excel_file, rb_results_lst)

'C:/Users/User/OneDrive - UVa/1_ASIGNATURAS/TFM/1_DataProcessing/CulebraPointsCBI_mlt_classed_bands_REP.xlsx'